In [1]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader

In [3]:


# Number of nodes in the graph
num_nodes = 4

# Number of features per node
num_features = 2

# Number of time steps
num_timesteps = 3

# Number of classes
num_classes = 2

# Number of graphs (time series)
num_graphs = 100

# Create a list to store graph data objects
data_list = []

# Generate random graphs
for _ in range(num_graphs):
    x = torch.randn(num_timesteps, num_nodes, num_features)
#     edge_index = torch.randint(num_nodes, (2, num_nodes))
    edge_index = torch.randint(num_nodes, (2, num_nodes * 2))
    y = torch.randint(num_classes, (1,)).item()

    data = Data(x=x, edge_index=edge_index, y=y)
    data_list.append(data)

# Create a DataLoader
loader = DataLoader(data_list, batch_size=32)

class STGNN(torch.nn.Module):
    def __init__(self):
        super(STGNN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_features)
        self.lin = Linear(num_features, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Apply temporal convolution
        for t in range(x.size(0)):
            x_t = x[t]
            x_t = self.conv1(x_t, edge_index)
            x_t = self.conv2(x_t, edge_index)
            x[t] = x_t

        # Perform a global mean pooling
        x = x.mean(dim=0)
        x = global_mean_pool(x, batch)

        # Classification layer
        x = self.lin(x)

        return F.log_softmax(x, dim=1)

# Initialize the model and optimizer
model = STGNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(100):
    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()

RuntimeError: index 5 is out of bounds for dimension 0 with size 4